In [1]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from prophet import Prophet
import datetime as dt
# import kagglehub as k
import json
from dotenv import load_dotenv
# from urllib3.exceptions import IncompleteRead
# from tenacity import retry, stop_after_attempt, wait_exponential

imdb_movies_by_genre_df = pd.read_csv('Resources/imdb_movies_by_genre_dataset.csv')
movies_box_office_2000_to_2024_df = pd.read_csv('Resources/movies_box_office_2000_to_2024.csv')
mojo_budget_update_df = pd.read_csv('Resources/Mojo_budget_update.csv')
movies_box_office_2000_to_2024_df = pd.read_csv('Resources/movies_box_office_2000_to_2024.csv')
mojo_budget_data_df = pd.read_csv('Resources/Mojo_budget_data.csv')
movies_data_df = pd.read_csv('Resources/movies_data.csv', encoding='ISO-8859-1') 

In [2]:
# load_dotenv()
# OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')



# lat, lon = 34.0522, -118.2437  # Los Angeles coordinates for example
# weather_data = []

# for date in pd.date_range("2023-01-01", "2023-01-03"):
#     unix_date = int(date.timestamp())
#     url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_date}&appid={OPENWEATHER_API_KEY}"
    

#     response = requests.get(url)
#     data = response.json()
#     weather_data.append(data)

#     print(weather_data)


In [3]:
# load_dotenv()
# CALENDARIFIC_API_KEY = os.getenv('CALENDARIFIC_API_KEY')

# # Set parameters
# country = "US"
# year = 2023

# # Construct the URL using an f-string
# url = f"https://calendarific.com/api/v2/holidays?api_key={CALENDARIFIC_API_KEY}&country={country}&year={year}"

# # Make the GET request
# response = requests.get(url)
# holidays_data = response.json().get('response', {}).get('holidays', [])

# # Convert the data to a DataFrame
# holidays_df = pd.DataFrame(holidays_data)

# # Extract ISO format dates and convert to datetime, handling timezone
# holidays_df['date'] = pd.to_datetime(holidays_df['date'].apply(lambda x: x['iso']), errors='coerce')

# # Keep only necessary columns
# holidays_df = holidays_df[['date', 'name']]
# holidays_df['is_holiday'] = True

# # Display the DataFrame
# print(holidays_df)

In [4]:
FOURSQUARE_API_KEY = 'FOURSQUARE_API_KEY'

FOURSQUARE_URL = 'https://api.foursquare.com/v3/places/search'

def get_theater_count(city):
    headers = {
        "Authorization": FOURSQUARE_API_KEY,
        "Accept": "application/json"
    }
    
    params = {
        "query": "cinema",
        "near": city,
        "limit": 50  # Limit of results (can be adjusted)
    }
    

    
    # Request theaters data from Foursquare
    response = requests.get(FOURSQUARE_URL, headers=headers, params=params)
    data = response.json()
    
    # Count the number of venues matching 'cinema'
    theater_count = len(data.get('results', []))
    return theater_count

# print("Foursquare API Response:", data)

In [5]:
# City population data (example data, adjust as needed)
city_population_data = {

    "New York": 8343728,
    "Los Angeles": 3871360,
    "Chicago": 2817048,
    "Houston": 2156357,
    "Seattle": 632002,
    "Phoenix": 1528848,
    "Miami": 398341,  
}

# Convert city population data to a DataFrame for reference, if needed
city_population_df = pd.DataFrame(list(city_population_data.items()), columns=["city", "population"])

def approximate_city_box_office_with_theaters(df, city):
    # Validate that the city exists in the population data
    if city not in city_population_data:
        raise ValueError(f"{city} population data is not available.")
    
    # Calculate city population factor
    city_population = city_population_data[city]
    total_population = sum(city_population_data.values())
    city_population_factor = city_population / total_population
    
    # Get theater count for the city from Foursquare API
    theater_count = get_theater_count(city)
    
    # Estimate revenue by adjusting with population factor and theater count
    estimated_revenue = []
    for _, row in df.iterrows():
        movie = row["movie_title"]
        domestic_revenue = row["Domestic Revenue"]
        
        # Adjust domestic revenue by both city population factor and theater count
        city_gross_estimate = domestic_revenue * city_population_factor / theater_count
        estimated_revenue.append({
            "movie": movie,
            "city": city,
            "theater_count": theater_count,
            "estimated_city_gross": city_gross_estimate
        })
    
    # Convert to DataFrame for better readability
    revenue_df = pd.DataFrame(estimated_revenue)
    return revenue_df


In [6]:

def approximate_city_box_office_with_theaters(df, city):
    # Ensure city population data is defined
    if city not in city_population_data:
        raise ValueError(f"{city} population data is not available.")
    
    # Calculate city population factor
    city_population = city_population_data[city]
    total_population = sum(city_population_data.values())
    city_population_factor = city_population / total_population
    print(f"City Population Factor for {city}: {city_population_factor:.3f}")  # Limited to 3 decimal places
    
    # Get theater count for the city from Foursquare API (or set manually if API fails)
    theater_count = get_theater_count(city) or 50  # Fallback if API fails
    print(f"Theater Count for {city}: {theater_count}")  # Debugging print
    
    # Estimate revenue by adjusting with population factor and theater count
    estimated_revenue = []
    for _, row in df.iterrows():
        movie = row["Movie Title"]
        domestic_revenue = row["Domestic Revenue ($)"]
        
        # Check for missing or invalid revenue data
        if pd.isna(domestic_revenue) or domestic_revenue == 0:
            estimated_revenue.append({
                "movie": movie,
                "city": city,
                "theater_count": theater_count,
                "estimated_city_gross": 0.0
            })
            continue
        
        # Adjust domestic revenue by both city population factor and theater count
        city_gross_estimate = domestic_revenue * city_population_factor / theater_count
        estimated_revenue.append({
            "movie": movie,
            "city": city,
            "theater_count": theater_count,
            "estimated_city_gross": city_gross_estimate
        })
    
    # Convert to DataFrame and format estimated_city_gross with commas and two decimal places
    revenue_df = pd.DataFrame(estimated_revenue)
    revenue_df['estimated_city_gross'] = revenue_df['estimated_city_gross'].apply(lambda x: f"${x:,.2f}")
    
    return revenue_df

In [7]:
# approximate_box_office_df = approximate_city_box_office_with_theaters(new_order_imdb_movies_df, "New York")
# display(approximate_box_office_df)


In [8]:
# print(new_order_imdb_movies_df.columns)

In [9]:
# print(new_order_imdb_movies_df["Domestic Revenue ($)"].head())

In [10]:
#Explore movies_data.csv file
movies_data_df


,Movie,Director,Running time,Actor 1,Actor 2,Actor 3,Genre,Budget,Box Office,Actors Box Office %,Director Box Office %,Earnings,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,Release year,IMDb score
0,13 Hours,Michael Bay,144,Toby Stephens,James Badge Dale,David Costabile,Action,50000000,69400000,50.00,69.23,19400000,0,0.0,2016,7.4
1,16 Blocks,Richard Donner,102,Bruce Willis,David Zayas,Sasha Roiz,Action,52000000,65000000,33.33,50.00,13000000,0,0.0,2006,6.6
2,17 Again,Burr Steers,102,Matthew Perry,Hunter Parrish,Thomas Lennon,Comedy,20000000,139000000,44.44,25.00,119000000,0,0.0,2009,6.4
3,1982,Tommy Oliver,90,Bokeem Woodbine,Ruby Dee,Quinton Aaron,Drama,1000000,2000000,50.00,100.00,1000000,0,0.0,2013,7.1
4,2 Fast 2 Furious,John Singleton,107,Paul Walker,Cole Hauser,Mo Gallini,Action,76000000,236000000,75.00,42.85,160000000,0,0.0,2003,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969,Z Storm,David Lam,92,Michael Wong,Louis Koo,Stephen Au,Action,7000000,17000000,100.00,100.00,10000000,0,0.0,2014,5.3
3970,Zack and Miri Make a Porno,Kevin Smith,101,Gerry Bednob,Jeff Anderson,Jennifer Schwalbach Smith,Comedy,24000000,42000000,66.67,41.66,18000000,0,0.0,2008,6.6
3971,Zero Effect,Jake Kasdan,116,Kim Dickens,Ryan O'Neal,Angela Featherstone,Comedy,5000000,2000000,33.33,50.00,-3000000,0,0.0,1998,7.0
3972,Zombieland,Ruben Fleischer,88,Emma Stone,Bill Murray,Derek Graf,Adventure,23600000,102000000,66.67,33.33,78400000,0,0.0,2009,7.7


In [11]:
movies_data_df.dtypes

Movie                                   object
Director                                object
Running time                             int64
Actor 1                                 object
Actor 2                                 object
Actor 3                                 object
Genre                                   object
Budget                                   int64
Box Office                               int64
Actors Box Office %                    float64
Director Box Office %                  float64
Earnings                                 int64
Oscar and Golden Globes nominations      int64
Oscar and Golden Globes awards         float64
Release year                             int64
IMDb score                             float64
dtype: object

In [12]:
movies_data_df.sort_values(by='Release year', ascending=False)

,Movie,Director,Running time,Actor 1,Actor 2,Actor 3,Genre,Budget,Box Office,Actors Box Office %,Director Box Office %,Earnings,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,Release year,IMDb score
0,13 Hours,Michael Bay,144,Toby Stephens,James Badge Dale,David Costabile,Action,50000000,69400000,50.00,69.23,19400000,0,0.0,2016,7.4
1833,Captain America: Civil War,Anthony Russo,147,Robert Downey Jr.,Scarlett Johansson,Chris Evans,Action,250000000,1153000000,100.00,50.00,903000000,5,2.0,2016,8.2
2706,I Am Wrath,Chuck Russell,92,Sam Trammell,Rebecca De Mornay,Amanda Schull,Action,18000000,9700000,22.22,50.00,-8300000,0,0.0,2016,5.2
2626,Hands of Stone,Jonathan Jakubowicz,105,Robert De Niro,Jurnee Smollett-Bell,Edgar Ramírez,Action,20000000,5600000,28.88,0.00,-14400000,0,0.0,2016,7.2
2594,God's Not Dead 2,Harold Cronk,120,Benjamin A. Onyango,Robin Givens,Maria Canals-Barrera,Drama,5000000,24000000,100.00,100.00,19000000,0,0.0,2016,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,It Happened One Night,Frank Capra,65,Claudette Colbert,Alan Hale,Walter Connolly,Comedy,325000,3500000,100.00,60.00,3175000,5,5.0,1934,8.2
3272,She Done Him Wrong,Lowell Sherman,66,Mae West,Gilbert Roland,Louise Beavers,Comedy,200000,2000000,100.00,100.00,1800000,1,0.0,1933,6.5
2000,42nd Street,Lloyd Bacon,89,Ginger Rogers,Dick Powell,George Brent,Comedy,439000,2500000,100.00,100.00,2061000,2,2.0,1933,7.7
2654,Hell's Angels,Howard Hughes,96,Jean Harlow,Marian Marsh,James Hall,Drama,3950000,8500000,100.00,100.00,4550000,1,0.0,1930,7.8


In [13]:
#Clean this dataset, then merge it with mojo to include 'Budget', 'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards', 'IMDb score'.
movies_data_df.columns

Index(['Movie', 'Director', 'Running time', 'Actor 1', 'Actor 2', 'Actor 3',
       'Genre', 'Budget', 'Box Office', 'Actors Box Office %',
       'Director Box Office %', 'Earnings',
       'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards',
       'Release year', 'IMDb score'],
      dtype='object')

In [14]:
movies_data_df.value_counts('Release year', ascending = False)

Release year
2006    191
2002    187
2014    184
2008    184
2009    180
       ... 
1948      1
1950      1
1955      1
1958      1
1929      1
Name: count, Length: 86, dtype: int64

In [15]:
movies_data_2000_to_2020_df = movies_data_df[(movies_data_df['Release year']>= 2000) & (movies_data_df['Release year'] <= 2020)]
movies_data_2000_to_2020_df

,Movie,Director,Running time,Actor 1,Actor 2,Actor 3,Genre,Budget,Box Office,Actors Box Office %,Director Box Office %,Earnings,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,Release year,IMDb score
0,13 Hours,Michael Bay,144,Toby Stephens,James Badge Dale,David Costabile,Action,50000000,69400000,50.00,69.23,19400000,0,0.0,2016,7.4
1,16 Blocks,Richard Donner,102,Bruce Willis,David Zayas,Sasha Roiz,Action,52000000,65000000,33.33,50.00,13000000,0,0.0,2006,6.6
2,17 Again,Burr Steers,102,Matthew Perry,Hunter Parrish,Thomas Lennon,Comedy,20000000,139000000,44.44,25.00,119000000,0,0.0,2009,6.4
3,1982,Tommy Oliver,90,Bokeem Woodbine,Ruby Dee,Quinton Aaron,Drama,1000000,2000000,50.00,100.00,1000000,0,0.0,2013,7.1
4,2 Fast 2 Furious,John Singleton,107,Paul Walker,Cole Hauser,Mo Gallini,Action,76000000,236000000,75.00,42.85,160000000,0,0.0,2003,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968,Youth in Revolt,Miguel Arteta,90,Steve Buscemi,Ari Graynor,Fred Willard,Comedy,18000000,19300000,66.67,75.00,1300000,0,0.0,2009,6.5
3969,Z Storm,David Lam,92,Michael Wong,Louis Koo,Stephen Au,Action,7000000,17000000,100.00,100.00,10000000,0,0.0,2014,5.3
3970,Zack and Miri Make a Porno,Kevin Smith,101,Gerry Bednob,Jeff Anderson,Jennifer Schwalbach Smith,Comedy,24000000,42000000,66.67,41.66,18000000,0,0.0,2008,6.6
3972,Zombieland,Ruben Fleischer,88,Emma Stone,Bill Murray,Derek Graf,Adventure,23600000,102000000,66.67,33.33,78400000,0,0.0,2009,7.7


In [16]:
#Index(['Movie', 'Director', 'Running time', 'Actor 1', 'Actor 2', 'Actor 3',
      #'Genre', 'Budget', 'Box Office', 'Actors Box Office %',
      #'Director Box Office %', 'Earnings',
      #'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards',
      #'Release year', 'IMDb score'],
      #dtype='object')     
       
filter_movies_data_2000_to_2020_df = movies_data_2000_to_2020_df[['Movie', 'Release year', 'Actor 1', 'Actor 2', 'Genre', 'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards', 'IMDb score']]
filter_movies_data_2000_to_2020_df


,Movie,Release year,Actor 1,Actor 2,Genre,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,IMDb score
0,13 Hours,2016,Toby Stephens,James Badge Dale,Action,0,0.0,7.4
1,16 Blocks,2006,Bruce Willis,David Zayas,Action,0,0.0,6.6
2,17 Again,2009,Matthew Perry,Hunter Parrish,Comedy,0,0.0,6.4
3,1982,2013,Bokeem Woodbine,Ruby Dee,Drama,0,0.0,7.1
4,2 Fast 2 Furious,2003,Paul Walker,Cole Hauser,Action,0,0.0,5.9
...,...,...,...,...,...,...,...,...
3968,Youth in Revolt,2009,Steve Buscemi,Ari Graynor,Comedy,0,0.0,6.5
3969,Z Storm,2014,Michael Wong,Louis Koo,Action,0,0.0,5.3
3970,Zack and Miri Make a Porno,2008,Gerry Bednob,Jeff Anderson,Comedy,0,0.0,6.6
3972,Zombieland,2009,Emma Stone,Bill Murray,Adventure,0,0.0,7.7


In [17]:
rename_movies_data_2000_to_2020_df = filter_movies_data_2000_to_2020_df.rename(columns={'Movie':'Movie Title', 'Release year':'Release Year', 'Actor 1':'Leading Actor', 'Actor 2':'Supporting Actor'})
rename_movies_data_2000_to_2020_df

,Movie Title,Release Year,Leading Actor,Supporting Actor,Genre,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,IMDb score
0,13 Hours,2016,Toby Stephens,James Badge Dale,Action,0,0.0,7.4
1,16 Blocks,2006,Bruce Willis,David Zayas,Action,0,0.0,6.6
2,17 Again,2009,Matthew Perry,Hunter Parrish,Comedy,0,0.0,6.4
3,1982,2013,Bokeem Woodbine,Ruby Dee,Drama,0,0.0,7.1
4,2 Fast 2 Furious,2003,Paul Walker,Cole Hauser,Action,0,0.0,5.9
...,...,...,...,...,...,...,...,...
3968,Youth in Revolt,2009,Steve Buscemi,Ari Graynor,Comedy,0,0.0,6.5
3969,Z Storm,2014,Michael Wong,Louis Koo,Action,0,0.0,5.3
3970,Zack and Miri Make a Porno,2008,Gerry Bednob,Jeff Anderson,Comedy,0,0.0,6.6
3972,Zombieland,2009,Emma Stone,Bill Murray,Adventure,0,0.0,7.7


In [18]:
rename_movies_data_2000_to_2020_df.isnull().sum()

Movie Title                            0
Release Year                           0
Leading Actor                          0
Supporting Actor                       0
Genre                                  0
Oscar and Golden Globes nominations    0
Oscar and Golden Globes awards         3
IMDb score                             0
dtype: int64

In [19]:
rename_movies_data_2000_to_2020_df.dropna().copy()

,Movie Title,Release Year,Leading Actor,Supporting Actor,Genre,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,IMDb score
0,13 Hours,2016,Toby Stephens,James Badge Dale,Action,0,0.0,7.4
1,16 Blocks,2006,Bruce Willis,David Zayas,Action,0,0.0,6.6
2,17 Again,2009,Matthew Perry,Hunter Parrish,Comedy,0,0.0,6.4
3,1982,2013,Bokeem Woodbine,Ruby Dee,Drama,0,0.0,7.1
4,2 Fast 2 Furious,2003,Paul Walker,Cole Hauser,Action,0,0.0,5.9
...,...,...,...,...,...,...,...,...
3968,Youth in Revolt,2009,Steve Buscemi,Ari Graynor,Comedy,0,0.0,6.5
3969,Z Storm,2014,Michael Wong,Louis Koo,Action,0,0.0,5.3
3970,Zack and Miri Make a Porno,2008,Gerry Bednob,Jeff Anderson,Comedy,0,0.0,6.6
3972,Zombieland,2009,Emma Stone,Bill Murray,Adventure,0,0.0,7.7


In [20]:
#Explore 'mojo_budget_update.csv'
mojo_budget_update_df.head()

,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
0,tt0099088,Back to the Future Part III,1990,"Stranded in 1955, Marty McFly learns about the...",NaN,May 25,1 hr 58 min,Universal Pictures,Robert Zemeckis,Robert Zemeckis,...,Thomas F. Wilson,40000000.0,88277583.0,157866667.0,246144250.0,Adventure,Comedy,Sci-Fi,Western,https://www.boxofficemojo.com/title/tt0099088/...
1,tt0099165,The Bonfire of the Vanities,1990,"After his mistress runs over a young teen, a W...",R,December 22,2 hr 5 min,Warner Bros.,Brian De Palma,Michael Cristofer,...,Kim Cattrall,47000000.0,15691192.0,NaN,15691192.0,Comedy,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0099165/...
2,tt0099348,Dances with Wolves,1990,"Lieutenant John Dunbar, assigned to a remote w...",NaN,November 9,3 hr 1 min,Orion Pictures,Kevin Costner,Michael Blake,...,Rodney A. Grant,22000000.0,184208848.0,240000000.0,424208848.0,Adventure,Drama,Western,NaN,https://www.boxofficemojo.com/title/tt0099348/...
3,tt0099422,Dick Tracy,1990,The comic strip detective finds his life vastl...,NaN,June 15,1 hr 45 min,Walt Disney Studios Motion Pictures,Warren Beatty,Chester Gould,...,Charlie Korsmo,47000000.0,103738726.0,59000000.0,162738726.0,Action,Comedy,Crime,Music,https://www.boxofficemojo.com/title/tt0099422/...
4,tt0099423,Die Hard 2,1990,John McClane attempts to avert disaster as rog...,NaN,July 6,2 hr 4 min,Twentieth Century Fox,Renny Harlin,Walter Wager,...,Reginald VelJohnson,70000000.0,117540947.0,122706486.0,240247433.0,Action,Thriller,NaN,NaN,https://www.boxofficemojo.com/title/tt0099423/...


In [21]:
mojo_budget_update_df.value_counts('year', ascending = False)

year
2010    174
2009    166
1999    160
2002    159
2005    158
2004    155
2000    147
2001    147
2011    146
2003    145
2006    143
2008    131
2013    126
2016    119
2014    119
2019    114
2015    113
2012    113
2017     99
2018     97
2007     94
1998     81
1997     70
1994     51
1991     47
1996     40
1995     38
1992     29
1993     29
1990     19
2020     14
Name: count, dtype: int64

In [22]:
mojo_budget_update_df.sort_values(by='year')


,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
0,tt0099088,Back to the Future Part III,1990,"Stranded in 1955, Marty McFly learns about the...",NaN,May 25,1 hr 58 min,Universal Pictures,Robert Zemeckis,Robert Zemeckis,...,Thomas F. Wilson,40000000.0,88277583.0,157866667.0,246144250.0,Adventure,Comedy,Sci-Fi,Western,https://www.boxofficemojo.com/title/tt0099088/...
18,tt0100802,Total Recall,1990,When a man goes for virtual vacation memories ...,NaN,June 1,1 hr 53 min,Sony Pictures Releasing,Paul Verhoeven,Philip K. Dick,...,Rachel Ticotin,65000000.0,119412921.0,141905000.0,261317921.0,Action,Sci-Fi,Thriller,NaN,https://www.boxofficemojo.com/title/tt0100802/...
17,tt0100758,Teenage Mutant Ninja Turtles,1990,Four teenage mutant ninja turtles emerge from ...,NaN,March 30,1 hr 33 min,New Line Cinema,Steve Barron,Kevin Eastman,...,David Forman,13500000.0,135265915.0,66700000.0,201965915.0,Action,Adventure,Comedy,Family,https://www.boxofficemojo.com/title/tt0100758/...
16,tt0100405,Pretty Woman,1990,A man in a legal but hurtful business needs an...,R,March 23,1 hr 59 min,Walt Disney Studios Motion Pictures,Garry Marshall,J.F. Lawton,...,Laura San Giacomo,14000000.0,178406268.0,285000000.0,463406268.0,Comedy,Romance,NaN,NaN,https://www.boxofficemojo.com/title/tt0100405/...
15,tt0100332,Paris Is Burning,1990,A chronicle of New York's drag scene in the 19...,R,August 9,1 hr 11 min,Miramax,Jennie Livingston,NaN,...,Paris Duprée,500000.0,3905297.0,NaN,3905297.0,Documentary,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt0100332/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534,tt1634106,Bloodshot,2020,"Ray Garrison, a slain soldier, is re-animated ...",PG-13,March 6,1 hr 49 min,Sony Pictures Releasing,Dave Wilson,Jeff Wadlow,...,Toby Kebbell,45000000.0,10021787.0,18407068.0,28428855.0,Action,Drama,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt1634106/...
2989,tt3794354,Sonic the Hedgehog,2020,"After discovering a small, blue, fast hedgehog...",PG,February 12,1 hr 39 min,Paramount Pictures,Jeff Fowler,Patrick Casey,...,Tika Sumpter,85000000.0,146066470.0,160700000.0,306766470.0,Action,Adventure,Comedy,Family,https://www.boxofficemojo.com/title/tt3794354/...
2433,tt1502397,Bad Boys for Life,2020,Miami detectives Mike Lowrey and Marcus Burnet...,R,January 15,2 hr 4 min,Sony Pictures Releasing,Adil El Arbi,Peter Craig,...,Alexander Ludwig,90000000.0,204417855.0,214656791.0,419074646.0,Action,Comedy,Crime,Thriller,https://www.boxofficemojo.com/title/tt1502397/...
3228,tt8244784,The Hunt,2020,Twelve strangers wake up in a clearing. They d...,R,March 13,1 hr 30 min,Universal Pictures,Craig Zobel,Nick Cuse,...,Wayne Duvall,14000000.0,5812500.0,700000.0,6512500.0,Action,Horror,Thriller,NaN,https://www.boxofficemojo.com/title/tt8244784/...


In [23]:
mojo_budget_update_df.dtypes

movie_id            object
title               object
year                 int64
trivia              object
mpaa                object
release_date        object
run_time            object
distributor         object
director            object
writer              object
producer            object
composer            object
cinematographer     object
main_actor_1        object
main_actor_2        object
main_actor_3        object
main_actor_4        object
budget             float64
domestic           float64
international      float64
worldwide          float64
genre_1             object
genre_2             object
genre_3             object
genre_4             object
html                object
dtype: object

In [24]:
mojo_budget_update_df.columns

Index(['movie_id', 'title', 'year', 'trivia', 'mpaa', 'release_date',
       'run_time', 'distributor', 'director', 'writer', 'producer', 'composer',
       'cinematographer', 'main_actor_1', 'main_actor_2', 'main_actor_3',
       'main_actor_4', 'budget', 'domestic', 'international', 'worldwide',
       'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html'],
      dtype='object')

In [25]:
mojo_budget_update_2000_to_2020_df = mojo_budget_update_df[(mojo_budget_update_df['year']>= 2000) & (mojo_budget_update_df['year'] <= 2020)]
mojo_budget_update_2000_to_2020_df.sort_values(by='year')

,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
671,tt0186151,Frequency,2000,An accidental cross-time radio link connects f...,PG-13,April 28,1 hr 58 min,New Line Cinema,Gregory Hoblit,Toby Emmerich,...,Elizabeth Mitchell,31000000.0,45010278.0,23095967.0,68106245.0,Crime,Drama,Mystery,Sci-Fi,https://www.boxofficemojo.com/title/tt0186151/...
659,tt0183523,Mission to Mars,2000,When the first manned mission to Mars meets wi...,PG,March 10,1 hr 54 min,Walt Disney Studios Motion Pictures,Brian De Palma,Lowell Cannon,...,Connie Nielsen,100000000.0,60883407.0,50100000.0,110983407.0,Adventure,Sci-Fi,Thriller,NaN,https://www.boxofficemojo.com/title/tt0183523/...
750,tt0212346,Miss Congeniality,2000,An F.B.I. Agent must go undercover in the Miss...,PG-13,December 22,1 hr 49 min,Warner Bros.,Donald Petrie,Marc Lawrence,...,Candice Bergen,45000000.0,106807667.0,105935053.0,212742720.0,Action,Comedy,Crime,Romance,https://www.boxofficemojo.com/title/tt0212346/...
662,tt0184858,Reindeer Games,2000,After assuming his dead cell-mate's identity t...,R,February 25,1 hr 44 min,Miramax,John Frankenheimer,Ehren Kruger,...,James Frain,42000000.0,23368995.0,8799975.0,32168970.0,Action,Adventure,Crime,Drama,https://www.boxofficemojo.com/title/tt0184858/...
663,tt0184894,Shanghai Noon,2000,A Chinese man travels to the Wild West to resc...,PG-13,May 26,1 hr 50 min,Walt Disney Studios Motion Pictures,Tom Dey,Miles Millar,...,Brandon Merrill,55000000.0,56937502.0,42336965.0,99274467.0,Action,Adventure,Comedy,Western,https://www.boxofficemojo.com/title/tt0184894/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,tt7545266,Like a Boss,2020,Two friends with very different ideals start a...,R,January 9,1 hr 23 min,Paramount Pictures,Miguel Arteta,Sam Pitman,...,Destiny Adams,29000000.0,22169514.0,7583629.0,29753143.0,Comedy,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt7545266/...
2433,tt1502397,Bad Boys for Life,2020,Miami detectives Mike Lowrey and Marcus Burnet...,R,January 15,2 hr 4 min,Sony Pictures Releasing,Adil El Arbi,Peter Craig,...,Alexander Ludwig,90000000.0,204417855.0,214656791.0,419074646.0,Action,Comedy,Crime,Thriller,https://www.boxofficemojo.com/title/tt1502397/...
3218,tt7798646,The Photograph,2020,A series of intertwining love stories set in t...,PG-13,February 14,1 hr 46 min,Universal Pictures,Stella Meghie,Stella Meghie,...,Issa Rae,16000000.0,20578185.0,75019.0,20653204.0,Drama,Romance,NaN,NaN,https://www.boxofficemojo.com/title/tt7798646/...
2989,tt3794354,Sonic the Hedgehog,2020,"After discovering a small, blue, fast hedgehog...",PG,February 12,1 hr 39 min,Paramount Pictures,Jeff Fowler,Patrick Casey,...,Tika Sumpter,85000000.0,146066470.0,160700000.0,306766470.0,Action,Adventure,Comedy,Family,https://www.boxofficemojo.com/title/tt3794354/...


In [26]:
#'movie_id', 'title', 'year', 'trivia', 'mpaa', 'release_date',
      # 'run_time', 'distributor', 'director', 'writer', 'producer', 'composer',
       #'cinematographer', 'main_actor_1', 'main_actor_2', 'main_actor_3',
       #'main_actor_4', 'budget', 'domestic', 'international', 'worldwide',
       #'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html'
       
filter_mojo_budget_update_2000_2020_df = mojo_budget_update_2000_to_2020_df[["movie_id", "title", "year", "main_actor_1", "main_actor_2", "domestic", "mpaa", "genre_1", "release_date"]]
filter_mojo_budget_update_2000_2020_df

,movie_id,title,year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
257,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407.0,PG-13,Drama,September 21
356,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564.0,NaN,Adventure,June 23
365,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080.0,PG-13,Action,July 6
368,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000.0,R,Biography,October 25
369,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566.0,R,Horror,October 19
...,...,...,...,...,...,...,...,...,...
3238,tt8772262,Midsommar,2019,Florence Pugh,Jack Reynor,27426361.0,R,Drama,July 3
3239,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234.0,PG-13,Comedy,November 21
3240,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109.0,R,Biography,March 29
3241,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847.0,PG,Comedy,November 7


In [27]:
rename_mojo_budget_update_2000_2020_df = filter_mojo_budget_update_2000_2020_df.rename(columns={'title':'movie_title', 'year':'movie_year'})
rename_mojo_budget_update_2000_2020_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
257,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407.0,PG-13,Drama,September 21
356,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564.0,NaN,Adventure,June 23
365,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080.0,PG-13,Action,July 6
368,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000.0,R,Biography,October 25
369,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566.0,R,Horror,October 19
...,...,...,...,...,...,...,...,...,...
3238,tt8772262,Midsommar,2019,Florence Pugh,Jack Reynor,27426361.0,R,Drama,July 3
3239,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234.0,PG-13,Comedy,November 21
3240,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109.0,R,Biography,March 29
3241,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847.0,PG,Comedy,November 7


In [28]:
rename_mojo_budget_update_2000_2020_df.isnull().sum()

movie_id          0
movie_title       0
movie_year        0
main_actor_1      0
main_actor_2      0
domestic         19
mpaa            101
genre_1           0
release_date      1
dtype: int64

In [29]:
rename_mojo_budget_update_2000_2020_df.dropna().copy()

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
257,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407.0,PG-13,Drama,September 21
365,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080.0,PG-13,Action,July 6
368,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000.0,R,Biography,October 25
369,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566.0,R,Horror,October 19
382,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,Elijah Wood,Ian McKellen,315544750.0,PG-13,Action,December 19
...,...,...,...,...,...,...,...,...,...
3238,tt8772262,Midsommar,2019,Florence Pugh,Jack Reynor,27426361.0,R,Drama,July 3
3239,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234.0,PG-13,Comedy,November 21
3240,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109.0,R,Biography,March 29
3241,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847.0,PG,Comedy,November 7


In [30]:
#Explore 'mojo_budget_data.csv' file
mojo_budget_data_df



,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
0,tt0118589,Glitter,2001,Vondie Curtis-Hall,Cheryl L. West,Laurence Mark,Terence Blanchard,Geoffrey Simpson,Mariah Carey,Eric Benét,...,4274407,997259,5271666,PG-13,1 hr 44 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0118589/...
1,tt0120630,Chicken Run,2000,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,106834564,118000000,224834564,NaN,1 hr 24 min,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
2,tt0120667,Fantastic Four,2005,Tim Story,Mark Frost,Avi Arad,John Ottman,Oliver Wood,Ioan Gruffudd,Michael Chiklis,...,154696080,178839854,333535934,PG-13,1 hr 46 min,Action,Adventure,Family,Fantasy,https://www.boxofficemojo.com/title/tt0120667/...
3,tt0120679,Frida,2002,Julie Taymor,Hayden Herrera,Lindsay Flickinger,Elliot Goldenthal,Rodrigo Prieto,Salma Hayek,Alfred Molina,...,25885000,30413474,56298474,R,2 hr 3 min,Biography,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0120679/...
4,tt0120681,From Hell,2001,Albert Hughes,Alan Moore,Jane Hamsher,Trevor Jones,Peter Deming,Johnny Depp,Heather Graham,...,31602566,42955549,74558115,R,2 hr 2 min,Horror,Mystery,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120681/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,tt8688634,21 Bridges,2019,Brian Kirk,Adam Mervis,Chadwick Boseman,Alex Belcher,Paul Cameron,Chadwick Boseman,Sienna Miller,...,28539757,21400000,49939757,R,1 hr 39 min,Action,Crime,Drama,Thriller,https://www.boxofficemojo.com/title/tt8688634/...
2472,tt8946378,Knives Out,2019,Rian Johnson,Rian Johnson,Ram Bergman,Nathan Johnson,Steve Yedlin,Daniel Craig,Chris Evans,...,165363234,146189956,311553190,PG-13,2 hr 11 min,Comedy,Crime,Drama,Mystery,https://www.boxofficemojo.com/title/tt8946378/...
2473,tt9024106,Unplanned,2019,Chuck Konzelman,Abby Johnson,Chris Jones,Stephen Blake Kanicka,Drew Maw,Ashley Bratcher,Brooks Ryan,...,19005109,2046806,21051915,R,1 hr 49 min,Biography,Drama,NaN,NaN,https://www.boxofficemojo.com/title/tt9024106/...
2474,tt9134216,Playing with Fire,2019,Andy Fickman,Dan Ewen,Todd Garner,Nathan Wang,Dean Semler,John Cena,Keegan-Michael Key,...,44451847,24179822,68631669,PG,1 hr 36 min,Comedy,Family,NaN,NaN,https://www.boxofficemojo.com/title/tt9134216/...


In [31]:
mojo_budget_data_df.value_counts('movie_year', ascending= False)

movie_year
2010    163
2002    151
2009    149
2004    142
2011    140
2001    138
2000    137
2005    137
2003    134
2006    126
2008    123
2013    122
2014    117
2016    117
2015    109
2012    104
2017     98
2018     92
2007     88
2019     75
2020     14
Name: count, dtype: int64

In [32]:
mojo_budget_data_df.sort_values(by= 'movie_year')

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
109,tt0185183,Battlefield Earth,2000,Roger Christian,Corey Mandell,Jonathan D. Krane,Elia Cmiral,Giles Nuttgens,John Travolta,Forest Whitaker,...,21471685,8253978,29725663,PG-13,1 hr 58 min,Action,Adventure,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt0185183/...
101,tt0183505,"Me, Myself & Irene",2000,Bobby Farrelly,Peter Farrelly,Bobby Farrelly,Lee Scott,Mark Irwin,Jim Carrey,Renée Zellweger,...,90570999,58700000,149270999,R,1 hr 56 min,Comedy,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt0183505/...
102,tt0183523,Mission to Mars,2000,Brian De Palma,Lowell Cannon,Tom Jacobson,Ennio Morricone,Stephen H. Burum,Tim Robbins,Gary Sinise,...,60883407,50100000,110983407,PG,1 hr 54 min,Adventure,Sci-Fi,Thriller,NaN,https://www.boxofficemojo.com/title/tt0183523/...
172,tt0210616,Center Stage,2000,Nicholas Hytner,Carol Heikkinen,Laurence Mark,George Fenton,Geoffrey Simpson,Amanda Schull,Ethan Stiefel,...,17200925,9185016,26385941,PG-13,1 hr 55 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0210616/...
171,tt0210567,Beautiful,2000,Sally Field,Jon Bernstein,John Bertolli,John Frizzell,Robert D. Yeoman,Minnie Driver,Hallie Eisenberg,...,3157348,12582,3169930,PG-13,1 hr 52 min,Comedy,Drama,NaN,NaN,https://www.boxofficemojo.com/title/tt0210567/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828,tt1634106,Bloodshot,2020,Dave Wilson,Jeff Wadlow,Toby Jaffe,Steve Jablonsky,Jacques Jouffret,Vin Diesel,Eiza González,...,10021787,19727667,29749454,PG-13,1 hr 49 min,Action,Drama,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt1634106/...
1310,tt0983946,Fantasy Island,2020,Jeff Wadlow,Jillian Jacobs,Jason Blum,Bear McCreary,Toby Oliver,Michael Peña,Maggie Q,...,26441782,22165774,48607556,PG-13,1 hr 49 min,Adventure,Fantasy,Horror,Mystery,https://www.boxofficemojo.com/title/tt0983946/...
2416,tt6673612,Dolittle,2020,Stephen Gaghan,Stephen Gaghan,Susan Downey,Danny Elfman,Guillermo Navarro,Robert Downey Jr.,Antonio Banderas,...,77047065,150861065,227908130,PG,1 hr 41 min,Adventure,Comedy,Family,Fantasy,https://www.boxofficemojo.com/title/tt6673612/...
1371,tt1051906,The Invisible Man,2020,Leigh Whannell,Leigh Whannell,Jason Blum,Benjamin Wallfisch,Stefan Duscio,Elisabeth Moss,Oliver Jackson-Cohen,...,64914050,59647175,124561225,R,2 hr 4 min,Horror,Mystery,Sci-Fi,Thriller,https://www.boxofficemojo.com/title/tt1051906/...


In [33]:
mojo_budget_data_df.dtypes

movie_id           object
movie_title        object
movie_year          int64
director           object
writer             object
producer           object
composer           object
cinematographer    object
main_actor_1       object
main_actor_2       object
main_actor_3       object
main_actor_4       object
budget              int64
domestic            int64
international       int64
worldwide           int64
mpaa               object
run_time           object
genre_1            object
genre_2            object
genre_3            object
genre_4            object
link               object
dtype: object

In [34]:
mojo_budget_data_df.value_counts('genre_1')

genre_1
Action         761
Comedy         620
Drama          369
Adventure      256
Crime          144
Horror         141
Biography      139
Animation       20
Documentary     10
Fantasy          6
Mystery          4
Family           2
Sci-Fi           2
Music            1
Romance          1
Name: count, dtype: int64

In [35]:
mojo_budget_data_df.columns

Index(['movie_id', 'movie_title', 'movie_year', 'director', 'writer',
       'producer', 'composer', 'cinematographer', 'main_actor_1',
       'main_actor_2', 'main_actor_3', 'main_actor_4', 'budget', 'domestic',
       'international', 'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'link'],
      dtype='object')

In [36]:
mojo_budget_data_2000_to_2020_df = mojo_budget_data_df[(mojo_budget_data_df['movie_year']>= 2000) & (mojo_budget_data_df['movie_year'] <= 2020)]
mojo_budget_data_2000_to_2020_df

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
0,tt0118589,Glitter,2001,Vondie Curtis-Hall,Cheryl L. West,Laurence Mark,Terence Blanchard,Geoffrey Simpson,Mariah Carey,Eric Benét,...,4274407,997259,5271666,PG-13,1 hr 44 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0118589/...
1,tt0120630,Chicken Run,2000,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,106834564,118000000,224834564,NaN,1 hr 24 min,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
2,tt0120667,Fantastic Four,2005,Tim Story,Mark Frost,Avi Arad,John Ottman,Oliver Wood,Ioan Gruffudd,Michael Chiklis,...,154696080,178839854,333535934,PG-13,1 hr 46 min,Action,Adventure,Family,Fantasy,https://www.boxofficemojo.com/title/tt0120667/...
3,tt0120679,Frida,2002,Julie Taymor,Hayden Herrera,Lindsay Flickinger,Elliot Goldenthal,Rodrigo Prieto,Salma Hayek,Alfred Molina,...,25885000,30413474,56298474,R,2 hr 3 min,Biography,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0120679/...
4,tt0120681,From Hell,2001,Albert Hughes,Alan Moore,Jane Hamsher,Trevor Jones,Peter Deming,Johnny Depp,Heather Graham,...,31602566,42955549,74558115,R,2 hr 2 min,Horror,Mystery,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120681/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,tt8688634,21 Bridges,2019,Brian Kirk,Adam Mervis,Chadwick Boseman,Alex Belcher,Paul Cameron,Chadwick Boseman,Sienna Miller,...,28539757,21400000,49939757,R,1 hr 39 min,Action,Crime,Drama,Thriller,https://www.boxofficemojo.com/title/tt8688634/...
2472,tt8946378,Knives Out,2019,Rian Johnson,Rian Johnson,Ram Bergman,Nathan Johnson,Steve Yedlin,Daniel Craig,Chris Evans,...,165363234,146189956,311553190,PG-13,2 hr 11 min,Comedy,Crime,Drama,Mystery,https://www.boxofficemojo.com/title/tt8946378/...
2473,tt9024106,Unplanned,2019,Chuck Konzelman,Abby Johnson,Chris Jones,Stephen Blake Kanicka,Drew Maw,Ashley Bratcher,Brooks Ryan,...,19005109,2046806,21051915,R,1 hr 49 min,Biography,Drama,NaN,NaN,https://www.boxofficemojo.com/title/tt9024106/...
2474,tt9134216,Playing with Fire,2019,Andy Fickman,Dan Ewen,Todd Garner,Nathan Wang,Dean Semler,John Cena,Keegan-Michael Key,...,44451847,24179822,68631669,PG,1 hr 36 min,Comedy,Family,NaN,NaN,https://www.boxofficemojo.com/title/tt9134216/...


In [37]:
filter_mojo_budget_data_2000_2020_df = mojo_budget_data_2000_to_2020_df[["movie_id", "movie_title", "movie_year", "main_actor_1", "main_actor_2", "domestic", "mpaa", "genre_1"]]
filter_mojo_budget_data_2000_2020_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama
1,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror
...,...,...,...,...,...,...,...,...
2471,tt8688634,21 Bridges,2019,Chadwick Boseman,Sienna Miller,28539757,R,Action
2472,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234,PG-13,Comedy
2473,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109,R,Biography
2474,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847,PG,Comedy


In [38]:
filter_mojo_budget_data_2000_2020_df.isnull().sum()

movie_id         0
movie_title      0
movie_year       0
main_actor_1     0
main_actor_2     0
domestic         0
mpaa            62
genre_1          0
dtype: int64

In [39]:
filter_mojo_budget_data_2000_2020_df.dropna().copy()

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror
5,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,Elijah Wood,Ian McKellen,315544750,PG-13,Action
...,...,...,...,...,...,...,...,...
2471,tt8688634,21 Bridges,2019,Chadwick Boseman,Sienna Miller,28539757,R,Action
2472,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234,PG-13,Comedy
2473,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109,R,Biography
2474,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847,PG,Comedy


In [40]:
#We need to merge the two datasets. 
merge_df = pd.merge(filter_mojo_budget_data_2000_2020_df, rename_mojo_budget_update_2000_2020_df)
merge_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama,September 21
1,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure,June 23
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action,July 6
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography,October 25
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror,October 19
...,...,...,...,...,...,...,...,...,...
2448,tt8688634,21 Bridges,2019,Chadwick Boseman,Sienna Miller,28539757,R,Action,November 21
2449,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234,PG-13,Comedy,November 21
2450,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109,R,Biography,March 29
2451,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847,PG,Comedy,November 7


In [41]:
merge_df.value_counts('movie_title')

movie_title
The Illusionist                    2
Fantastic Four                     2
The Girl with the Dragon Tattoo    2
No Good Deed                       2
Hellboy                            2
                                  ..
Horrible Bosses                    1
Horrible Bosses 2                  1
Horton Hears a Who!                1
Hostel                             1
Æon Flux                           1
Name: count, Length: 2442, dtype: int64

In [42]:
merge_df.value_counts('genre_1')

genre_1
Action         753
Comedy         616
Drama          365
Adventure      254
Crime          144
Biography      139
Horror         137
Animation       19
Documentary     10
Fantasy          6
Mystery          4
Family           2
Sci-Fi           2
Music            1
Romance          1
Name: count, dtype: int64

In [43]:
merge_df.head()

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama,September 21
1,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure,June 23
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action,July 6
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography,October 25
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror,October 19


In [44]:
	# Rename 'main_actor_1' to 'Main Actor'
    # 'main_actor_2' to 'Supporting Actor'	
    # 'domestic' to 'Domestic'	
    # 'mpaa' to 'Film rating'	
    # 'genre_1' to 'Genre'	
    # 'title' to 'Movie title'	
    # 'year' to 'Year'	
    # 'release_date' to 'Release Date'
       
merge_clean_df = merge_df.rename(columns={'main_actor_1' : 'Leading Actor',
   'main_actor_2' : 'Supporting Actor',	
   'domestic' : 'Domestic Revenue ($)',	
   'mpaa' : 'Film Rating',	
   'genre_1' : 'Genre',	
   'movie_title' : 'Movie Title',	
   'movie_year' : 'Release Year',	
   'release_date' : 'Release Date',
   'movie_id':'Movie ID'})

merge_clean_df

,Movie ID,Movie Title,Release Year,Leading Actor,Supporting Actor,Domestic Revenue ($),Film Rating,Genre,Release Date
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama,September 21
1,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure,June 23
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action,July 6
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography,October 25
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror,October 19
...,...,...,...,...,...,...,...,...,...
2448,tt8688634,21 Bridges,2019,Chadwick Boseman,Sienna Miller,28539757,R,Action,November 21
2449,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234,PG-13,Comedy,November 21
2450,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109,R,Biography,March 29
2451,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847,PG,Comedy,November 7


In [45]:
merge_clean_df = merge_clean_df.dropna(subset=['Film Rating'])
merge_clean_df

,Movie ID,Movie Title,Release Year,Leading Actor,Supporting Actor,Domestic Revenue ($),Film Rating,Genre,Release Date
0,tt0118589,Glitter,2001,Mariah Carey,Eric Benét,4274407,PG-13,Drama,September 21
2,tt0120667,Fantastic Four,2005,Ioan Gruffudd,Michael Chiklis,154696080,PG-13,Action,July 6
3,tt0120679,Frida,2002,Salma Hayek,Alfred Molina,25885000,R,Biography,October 25
4,tt0120681,From Hell,2001,Johnny Depp,Heather Graham,31602566,R,Horror,October 19
5,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,Elijah Wood,Ian McKellen,315544750,PG-13,Action,December 19
...,...,...,...,...,...,...,...,...,...
2448,tt8688634,21 Bridges,2019,Chadwick Boseman,Sienna Miller,28539757,R,Action,November 21
2449,tt8946378,Knives Out,2019,Daniel Craig,Chris Evans,165363234,PG-13,Comedy,November 21
2450,tt9024106,Unplanned,2019,Ashley Bratcher,Brooks Ryan,19005109,R,Biography,March 29
2451,tt9134216,Playing with Fire,2019,John Cena,Keegan-Michael Key,44451847,PG,Comedy,November 7


In [46]:
merge_clean_df.isnull().sum()

Movie ID                0
Movie Title             0
Release Year            0
Leading Actor           0
Supporting Actor        0
Domestic Revenue ($)    0
Film Rating             0
Genre                   0
Release Date            0
dtype: int64

In [47]:
merge_new_order = ['Movie Title', 'Release Date', 'Release Year', 'Domestic Revenue ($)', 'Genre', 'Leading Actor', 'Supporting Actor', 'Film Rating']
new_mojo_merge_df = merge_clean_df[merge_new_order]
new_mojo_merge_df

,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13
2,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13
3,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R
4,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R
5,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13
...,...,...,...,...,...,...,...,...
2448,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R
2449,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13
2450,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R
2451,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG


In [48]:
new_mojo_merge_df.to_csv('merged_mojo_dataset.csv', index=False)

In [49]:
#Merge new_mojo_merge_df with rename_movies_data_2000_to_2020_df
merge_mojo_movies_data_df = pd.merge(new_mojo_merge_df, rename_movies_data_2000_to_2020_df[['Movie Title', 'Release Year', 'Leading Actor', 'Supporting Actor', 'Genre', 'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards', 'IMDb score']], on=['Movie Title', 'Release Year', 'Genre', 'Leading Actor', 'Supporting Actor'], how='left')
merge_mojo_movies_data_df


,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,IMDb score
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13,NaN,NaN,NaN
1,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,NaN,NaN,NaN
2,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R,NaN,NaN,NaN
3,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R,NaN,NaN,NaN
4,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2390,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R,NaN,NaN,NaN
2391,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,NaN,NaN,NaN
2392,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,NaN,NaN,NaN
2393,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG,NaN,NaN,NaN


In [50]:
merge_mojo_movies_data_df.isnull().sum()

Movie Title                               0
Release Date                              0
Release Year                              0
Domestic Revenue ($)                      0
Genre                                     0
Leading Actor                             0
Supporting Actor                          0
Film Rating                               0
Oscar and Golden Globes nominations    2311
Oscar and Golden Globes awards         2311
IMDb score                             2311
dtype: int64

In [51]:
merge_mojo_movies_data_clean_df = merge_mojo_movies_data_df.drop(['Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards', 'IMDb score'], axis=1)
merge_mojo_movies_data_clean_df

,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13
1,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13
2,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R
3,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R
4,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13
...,...,...,...,...,...,...,...,...
2390,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R
2391,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13
2392,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R
2393,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG


In [52]:
# #Merge new_mojo_merge_df with rename_movies_data_2000_to_2020_df
common_movie_titles_df = new_mojo_merge_df['Movie Title'].isin(rename_movies_data_2000_to_2020_df['Movie Title'])
sum(common_movie_titles_df)


1760

In [53]:
merge_mojo_movies_data_clean_df.isnull().sum()

Movie Title             0
Release Date            0
Release Year            0
Domestic Revenue ($)    0
Genre                   0
Leading Actor           0
Supporting Actor        0
Film Rating             0
dtype: int64

imdb_movies_by_genre_df cleaning

In [54]:
imdb_movies_by_genre_df

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35847,tt26738164,Pain Killer Kings,NaN,NaN,NaN,"Action, Crime, Drama",NaN,Add a Plot,Jeff Profitt,/name/nm0698489/,Jeff Profitt,/name/nm0698489/,NaN,NaN
35848,tt26738615,Joker Joker,2023,NaN,NaN,"Action, Crime, Romance",NaN,A struggling female writer goes to a estate to...,Rajesh Murthy,/name/nm9235197/,"Ankita Bhattacharya, \nRahul Somanna, \nPrabhesh","/name/nm11470111/,/name/nm11320206/,/name/nm14...",NaN,NaN
35849,tt26739274,Female Tazza,2021,NaN,96 min,"Crime, Drama",NaN,Add a Plot,Ji-seung Lee,/name/nm2360790/,"Lee Chae-young, \nChoi Chul Min, \nKim Sa Hee,...","/name/nm13600173/,/name/nm11192964/,/name/nm10...",NaN,NaN
35850,tt26739517,Dead in the Water,V,NaN,NaN,"Crime, Mystery, Thriller",NaN,A very wealthy heiress wakes up to find your h...,Rick Mowat,/name/nm0610250/,NaN,NaN,NaN,NaN


In [55]:
# return a count of how many movies there are from each year, sorted by the number of occurrences in descending order
imdb_movies_by_genre_df.value_counts(
    'year', ascending= False), imdb_movies_by_genre_df.sort_values(by= 'year')
imdb_movies_by_genre_df


,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35847,tt26738164,Pain Killer Kings,NaN,NaN,NaN,"Action, Crime, Drama",NaN,Add a Plot,Jeff Profitt,/name/nm0698489/,Jeff Profitt,/name/nm0698489/,NaN,NaN
35848,tt26738615,Joker Joker,2023,NaN,NaN,"Action, Crime, Romance",NaN,A struggling female writer goes to a estate to...,Rajesh Murthy,/name/nm9235197/,"Ankita Bhattacharya, \nRahul Somanna, \nPrabhesh","/name/nm11470111/,/name/nm11320206/,/name/nm14...",NaN,NaN
35849,tt26739274,Female Tazza,2021,NaN,96 min,"Crime, Drama",NaN,Add a Plot,Ji-seung Lee,/name/nm2360790/,"Lee Chae-young, \nChoi Chul Min, \nKim Sa Hee,...","/name/nm13600173/,/name/nm11192964/,/name/nm10...",NaN,NaN
35850,tt26739517,Dead in the Water,V,NaN,NaN,"Crime, Mystery, Thriller",NaN,A very wealthy heiress wakes up to find your h...,Rick Mowat,/name/nm0610250/,NaN,NaN,NaN,NaN


In [56]:
imdb_movies_by_genre_df.isnull().sum()

movie_id           0
movie_name         0
year            3563
certificate    24626
runtime         8220
genre              0
rating         11512
description        0
director        1601
director_id     1601
star            3534
star_id         3151
votes          11512
gross(in $)    33226
dtype: int64

In [57]:
# Drop rows with NaN values in 'certificate' and 'runtime' columns
imdb_movies_by_genre_df = imdb_movies_by_genre_df.dropna(subset=['certificate', 'runtime','rating', 'director'])

# Convert 'year' to integer after removing NaN values
imdb_movies_by_genre_df['year'] = imdb_movies_by_genre_df['year'].astype(int)

# Filter the DataFrame for years between 2000 and 2023
filtered_imdb_movies_by_genre_df = imdb_movies_by_genre_df[(imdb_movies_by_genre_df['year'] >= 2000) & (imdb_movies_by_genre_df['year'] <= 2023)]

# Display the result
filtered_imdb_movies_by_genre_df



/var/folders/ts/dsv00vbn2ygg_1mgkl5xxg_c0000gn/T/ipykernel_5245/3703758680.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_movies_by_genre_df['year'] = imdb_movies_by_genre_df['year'].astype(int)


,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN
5,tt1877830,The Batman,2022,PG-13,176 min,"Action, Crime, Drama",7.8,When a sadistic serial killer begins murdering...,Matt Reeves,/name/nm0716257/,"Robert Pattinson, \nZoë Kravitz, \nJeffrey Wri...","/name/nm1500155/,/name/nm2368789/,/name/nm0942...",672146.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35029,tt5126250,The Necklace,2015,R,83 min,"Crime, Drama, Mystery",7.4,When a reluctant scam artist's plan to unload ...,"David Chow, \nRudra Bahadur Gurung",/name/nm7663173/,"Dipti Gurung, \nRoshni Karki, \nR.C. Manoj, \n...","/name/nm7663164/,/name/nm7663166/,/name/nm7663...",7.0,NaN
35219,tt1303898,Shadow of Crime,2009,Not Rated,108 min,"Crime, Drama",7.6,Charles and Raymond are sent to Detroit to hit...,Richard G. James,/name/nm3157118/,"John Anton, \nKenny Clark, \nJoe Comaianni, \n...","/name/nm1123696/,/name/nm3920456/,/name/nm3169...",19.0,NaN
35223,tt3202828,Before Dawn Cracks,2007,Not Rated,126 min,"Crime, Drama, Mystery",5.2,"Before Dawn Cracks, as the title suggests, is ...",Hoi Kuok Meng,/name/nm5959596/,"Howard Chan, \nElisa Cheang, \nXianling Chen, ...","/name/nm1382337/,/name/nm5959598/,/name/nm5081...",73.0,NaN
35281,tt5077150,A Young King Must Die,2016,R,100 min,"Crime, Drama, History",3.6,"A DocuDrama exploring the facts, theories, and...",Alexander Roman,/name/nm3486688/,"Cyrille Autin, \nKevin Wayne Berger, \nAri Dav...","/name/nm1443247/,/name/nm3998859/,/name/nm1517...",7.0,NaN


In [58]:
filtered_imdb_movies_by_genre_df.isnull().sum()


movie_id          0
movie_name        0
year              0
certificate       0
runtime           0
genre             0
rating            0
description       0
director          0
director_id       0
star              4
star_id           1
votes             0
gross(in $)    3241
dtype: int64

In [59]:
filtered_imdb_movies_by_genre_df.dropna().copy()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nChris Evans, \nAna de Armas, \...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622.0,165359751.0
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, \nJonah Hill, \nMargot Robb...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476.0,116900694.0
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, \nHeath Ledger, \nAaron Eckhar...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531.0,534858444.0
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, \nJustin Theroux, \nJosh Lucas...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221.0,15070285.0
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, \nMichael Nyqvist, \nAlfie Allen...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257.0,43037835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, \nIrène Jacob, \nBenjamin Rolla...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139.0,10529.0
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, \nPrakash Raj, \nVimala Raman, ...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126.0,4550.0
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, \nRich Komenich, \nCharles Napier...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94.0,438906.0
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, \nDeliza Alcozer, \nMildred...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33.0,1500.0


In [60]:
filtered_imdb_movies_by_genre_df.isnull().sum()


movie_id          0
movie_name        0
year              0
certificate       0
runtime           0
genre             0
rating            0
description       0
director          0
director_id       0
star              4
star_id           1
votes             0
gross(in $)    3241
dtype: int64

In [61]:
filtered_imdb_movies_by_genre_df = filtered_imdb_movies_by_genre_df.dropna(subset=['certificate'])
filtered_imdb_movies_by_genre_df.isnull().sum()

movie_id          0
movie_name        0
year              0
certificate       0
runtime           0
genre             0
rating            0
description       0
director          0
director_id       0
star              4
star_id           1
votes             0
gross(in $)    3241
dtype: int64

In [62]:
filtered_imdb_movies_by_genre_df.reset_index(drop=True)
filtered_imdb_movies_by_genre_df = filtered_imdb_movies_by_genre_df.dropna(subset=['votes','gross(in $)'])
filtered_imdb_movies_by_genre_df

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nChris Evans, \nAna de Armas, \...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622.0,165359751.0
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, \nJonah Hill, \nMargot Robb...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476.0,116900694.0
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, \nHeath Ledger, \nAaron Eckhar...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531.0,534858444.0
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, \nJustin Theroux, \nJosh Lucas...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221.0,15070285.0
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, \nMichael Nyqvist, \nAlfie Allen...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257.0,43037835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, \nIrène Jacob, \nBenjamin Rolla...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139.0,10529.0
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, \nPrakash Raj, \nVimala Raman, ...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126.0,4550.0
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, \nRich Komenich, \nCharles Napier...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94.0,438906.0
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, \nDeliza Alcozer, \nMildred...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33.0,1500.0


In [63]:
#Rename columns
filtered_imdb_movies_by_genre_df = filtered_imdb_movies_by_genre_df.rename(columns={'movie_name':'Movie Title', 'year':'Year', 'certificate': "Film Rating", 'runtime':'Runtime', 'star':'Main Actor', 'director':'Director', 'genre':'Genre', 'votes':'Votes', 'gross(in $)':'Domestic Revenue ($)'})
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nChris Evans, \nAna de Armas, \...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622.0,165359751.0
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, \nJonah Hill, \nMargot Robb...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476.0,116900694.0
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, \nHeath Ledger, \nAaron Eckhar...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531.0,534858444.0
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, \nJustin Theroux, \nJosh Lucas...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221.0,15070285.0
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, \nMichael Nyqvist, \nAlfie Allen...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257.0,43037835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, \nIrène Jacob, \nBenjamin Rolla...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139.0,10529.0
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, \nPrakash Raj, \nVimala Raman, ...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126.0,4550.0
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, \nRich Komenich, \nCharles Napier...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94.0,438906.0
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, \nDeliza Alcozer, \nMildred...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33.0,1500.0


In [64]:
# Define a function to clean the 'Main Actor' data
def clean_main_actor(actor_string):
    return actor_string.replace('\n', ',')
filtered_imdb_movies_by_genre_df['Main Actor'] = filtered_imdb_movies_by_genre_df['Main Actor'].apply(clean_main_actor)
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622.0,165359751.0
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476.0,116900694.0
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531.0,534858444.0
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, ,Justin Theroux, ,Josh Lucas, ...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221.0,15070285.0
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257.0,43037835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139.0,10529.0
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126.0,4550.0
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94.0,438906.0
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, ,Deliza Alcozer, ,Mildred A...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33.0,1500.0


In [65]:
filtered_imdb_movies_by_genre_df.isnull().sum()

movie_id                0
Movie Title             0
Year                    0
Film Rating             0
Runtime                 0
Genre                   0
rating                  0
description             0
Director                0
director_id             0
Main Actor              0
star_id                 0
Votes                   0
Domestic Revenue ($)    0
dtype: int64

In [66]:
filtered_imdb_movies_by_genre_df = filtered_imdb_movies_by_genre_df.dropna(subset=['Film Rating'])
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622.0,165359751.0
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476.0,116900694.0
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531.0,534858444.0
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, ,Justin Theroux, ,Josh Lucas, ...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221.0,15070285.0
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257.0,43037835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139.0,10529.0
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126.0,4550.0
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94.0,438906.0
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, ,Deliza Alcozer, ,Mildred A...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33.0,1500.0


In [67]:
filtered_imdb_movies_by_genre_df.isnull().sum()

movie_id                0
Movie Title             0
Year                    0
Film Rating             0
Runtime                 0
Genre                   0
rating                  0
description             0
Director                0
director_id             0
Main Actor              0
star_id                 0
Votes                   0
Domestic Revenue ($)    0
dtype: int64

In [68]:
#Rounding and converting to integer
filtered_imdb_movies_by_genre_df [['Domestic Revenue ($)', 'Votes']] = filtered_imdb_movies_by_genre_df[['Domestic Revenue ($)', 'Votes']].round().astype(int)
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($)
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,"Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...","/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622,165359751
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476,116900694
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,"Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...","/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531,534858444
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,"Christian Bale, ,Justin Theroux, ,Josh Lucas, ...","/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221,15070285
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,"Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...","/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257,43037835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,"Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...","/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139,10529
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,"Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...","/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126,4550
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,"Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...","/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94,438906
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,"Adriana Carradero, ,Deliza Alcozer, ,Mildred A...","/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33,1500


In [69]:
new_order_imdb_movies_df = ['Movie Title', 'Year', 'Domestic Revenue ($)', 'Genre', 'Main Actor', 'Director', 'Film Rating', 'Runtime', 'Votes']
new_order_imdb_movies_df = filtered_imdb_movies_by_genre_df[new_order_imdb_movies_df]
new_order_imdb_movies_df.reset_index(drop=True)
new_order_imdb_movies_df

,Movie Title,Year,Domestic Revenue ($),Genre,Main Actor,Director,Film Rating,Runtime,Votes
7,Knives Out,2019,165359751,"Comedy, Crime, Drama","Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...",Rian Johnson,PG-13,130 min,705622
9,The Wolf of Wall Street,2013,116900694,"Biography, Comedy, Crime","Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...",Martin Scorsese,R,180 min,1430476
13,The Dark Knight,2008,534858444,"Action, Crime, Drama","Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...",Christopher Nolan,PG-13,152 min,2675531
14,American Psycho,2000,15070285,"Crime, Drama, Horror","Christian Bale, ,Justin Theroux, ,Josh Lucas, ...",Mary Harron,R,102 min,636221
21,John Wick,2014,43037835,"Action, Crime, Thriller","Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...","Chad Stahelski, \nDavid Leitch",R,101 min,644257
...,...,...,...,...,...,...,...,...,...
18182,Autumn,2004,10529,"Crime, Drama","Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...",Ra'up McGee,Unrated,110 min,139
18320,Mumbai Mirror,2013,4550,"Action, Crime, Thriller","Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...",Ankush Bhatt,Not Rated,136 min,126
18402,Forgive Me Father,2001,438906,"Crime, Drama, Mystery","Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...",Ivan Rogers,R,138 min,94
23621,I Recorded a Murder!,2016,1500,"Crime, Drama, Horror","Adriana Carradero, ,Deliza Alcozer, ,Mildred A...",Manny Velazquez,Not Rated,100 min,33


In [70]:
# Reset index
new_order_imdb_movies_df = new_order_imdb_movies_df.reset_index(drop=True)
new_order_imdb_movies_df.index = new_order_imdb_movies_df.index + 1
new_order_imdb_movies_df

,Movie Title,Year,Domestic Revenue ($),Genre,Main Actor,Director,Film Rating,Runtime,Votes
1,Knives Out,2019,165359751,"Comedy, Crime, Drama","Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...",Rian Johnson,PG-13,130 min,705622
2,The Wolf of Wall Street,2013,116900694,"Biography, Comedy, Crime","Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...",Martin Scorsese,R,180 min,1430476
3,The Dark Knight,2008,534858444,"Action, Crime, Drama","Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...",Christopher Nolan,PG-13,152 min,2675531
4,American Psycho,2000,15070285,"Crime, Drama, Horror","Christian Bale, ,Justin Theroux, ,Josh Lucas, ...",Mary Harron,R,102 min,636221
5,John Wick,2014,43037835,"Action, Crime, Thriller","Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...","Chad Stahelski, \nDavid Leitch",R,101 min,644257
...,...,...,...,...,...,...,...,...,...
1346,Autumn,2004,10529,"Crime, Drama","Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...",Ra'up McGee,Unrated,110 min,139
1347,Mumbai Mirror,2013,4550,"Action, Crime, Thriller","Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...",Ankush Bhatt,Not Rated,136 min,126
1348,Forgive Me Father,2001,438906,"Crime, Drama, Mystery","Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...",Ivan Rogers,R,138 min,94
1349,I Recorded a Murder!,2016,1500,"Crime, Drama, Horror","Adriana Carradero, ,Deliza Alcozer, ,Mildred A...",Manny Velazquez,Not Rated,100 min,33


In [71]:
# Clean Main Actor
print(new_order_imdb_movies_df['Main Actor'])

1       Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Ja...
2       Leonardo DiCaprio, ,Jonah Hill, ,Margot Robbie...
3       Christian Bale, ,Heath Ledger, ,Aaron Eckhart,...
4       Christian Bale, ,Justin Theroux, ,Josh Lucas, ...
5       Keanu Reeves, ,Michael Nyqvist, ,Alfie Allen, ...
                              ...                        
1346    Laurent Lucas, ,Irène Jacob, ,Benjamin Rolland...
1347    Sachiin Joshi, ,Prakash Raj, ,Vimala Raman, ,M...
1348    Ivan Rogers, ,Rich Komenich, ,Charles Napier, ...
1349    Adriana Carradero, ,Deliza Alcozer, ,Mildred A...
1350    Claudia Dzienny, ,Prudence Vindin, ,Peter J. W...
Name: Main Actor, Length: 1350, dtype: object


In [72]:
first_main_actor = new_order_imdb_movies_df['Main Actor'].iloc[0]
print(first_main_actor)

Daniel Craig, ,Chris Evans, ,Ana de Armas, ,Jamie Lee Curtis


In [73]:
filtered_imdb_movies_by_genre_df[['Main Actor', 'actor_2', 'actor_3', 'actor_4']] = filtered_imdb_movies_by_genre_df['Main Actor'].str.split(', ', expand=True)
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($),actor_2,actor_3,actor_4
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,Daniel Craig,"/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622,165359751,",Chris Evans",",Ana de Armas",",Jamie Lee Curtis"
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,Leonardo DiCaprio,"/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476,116900694,",Jonah Hill",",Margot Robbie",",Matthew McConaughey"
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,Christian Bale,"/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531,534858444,",Heath Ledger",",Aaron Eckhart",",Michael Caine"
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,Christian Bale,"/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221,15070285,",Justin Theroux",",Josh Lucas",",Bill Sage"
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,Keanu Reeves,"/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257,43037835,",Michael Nyqvist",",Alfie Allen",",Willem Dafoe"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,Laurent Lucas,"/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139,10529,",Irène Jacob",",Benjamin Rolland",",Dinara Drukarova"
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,Sachiin Joshi,"/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126,4550,",Prakash Raj",",Vimala Raman",",Mukesh Agrohari"
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,Ivan Rogers,"/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94,438906,",Rich Komenich",",Charles Napier",",Chris Elbert"
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,Adriana Carradero,"/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33,1500,",Deliza Alcozer",",Mildred Amador",",Naudia Brown"


In [74]:
#Rename actor_2 to Supporting Actor
filtered_imdb_movies_by_genre_df = filtered_imdb_movies_by_genre_df.rename(columns={'actor_2':'Supporting Actor'})
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($),Supporting Actor,actor_3,actor_4
7,tt8946378,Knives Out,2019,PG-13,130 min,"Comedy, Crime, Drama",7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,Daniel Craig,"/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622,165359751,",Chris Evans",",Ana de Armas",",Jamie Lee Curtis"
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,Leonardo DiCaprio,"/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476,116900694,",Jonah Hill",",Margot Robbie",",Matthew McConaughey"
13,tt0468569,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,Christian Bale,"/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531,534858444,",Heath Ledger",",Aaron Eckhart",",Michael Caine"
14,tt0144084,American Psycho,2000,R,102 min,"Crime, Drama, Horror",7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,Christian Bale,"/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221,15070285,",Justin Theroux",",Josh Lucas",",Bill Sage"
21,tt2911666,John Wick,2014,R,101 min,"Action, Crime, Thriller",7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,Keanu Reeves,"/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257,43037835,",Michael Nyqvist",",Alfie Allen",",Willem Dafoe"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,"Crime, Drama",6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,Laurent Lucas,"/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139,10529,",Irène Jacob",",Benjamin Rolland",",Dinara Drukarova"
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,"Action, Crime, Thriller",5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,Sachiin Joshi,"/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126,4550,",Prakash Raj",",Vimala Raman",",Mukesh Agrohari"
18402,tt0245904,Forgive Me Father,2001,R,138 min,"Crime, Drama, Mystery",5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,Ivan Rogers,"/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94,438906,",Rich Komenich",",Charles Napier",",Chris Elbert"
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,"Crime, Drama, Horror",6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,Adriana Carradero,"/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33,1500,",Deliza Alcozer",",Mildred Amador",",Naudia Brown"


In [75]:
#Clean Supporting Actor
def clean_supporting_actor(supporting_actor_string):
    return supporting_actor_string.replace(',', '')
filtered_imdb_movies_by_genre_df['Supporting Actor'] = filtered_imdb_movies_by_genre_df['Supporting Actor'].apply(clean_supporting_actor)

In [76]:
print(filtered_imdb_movies_by_genre_df['Genre'])

7            Comedy, Crime, Drama
9        Biography, Comedy, Crime
13           Action, Crime, Drama
14           Crime, Drama, Horror
21        Action, Crime, Thriller
                   ...           
18182                Crime, Drama
18320     Action, Crime, Thriller
18402       Crime, Drama, Mystery
23621        Crime, Drama, Horror
24745      Comedy, Crime, Mystery
Name: Genre, Length: 1350, dtype: object


In [77]:
#c Clean Genre column
Genre_1 = filtered_imdb_movies_by_genre_df['Genre'].iloc[0]
print(Genre_1)

Comedy, Crime, Drama


In [78]:
filtered_imdb_movies_by_genre_df[['Genre', 'Genre 2', 'Genre 3']] = filtered_imdb_movies_by_genre_df['Genre'].str.split(', ', expand=True)
filtered_imdb_movies_by_genre_df

,movie_id,Movie Title,Year,Film Rating,Runtime,Genre,rating,description,Director,director_id,Main Actor,star_id,Votes,Domestic Revenue ($),Supporting Actor,actor_3,actor_4,Genre 2,Genre 3
7,tt8946378,Knives Out,2019,PG-13,130 min,Comedy,7.9,A detective investigates the death of the patr...,Rian Johnson,/name/nm0426059/,Daniel Craig,"/name/nm0185819/,/name/nm0262635/,/name/nm1869...",705622,165359751,Chris Evans,",Ana de Armas",",Jamie Lee Curtis",Crime,Drama
9,tt0993846,The Wolf of Wall Street,2013,R,180 min,Biography,8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,Leonardo DiCaprio,"/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1430476,116900694,Jonah Hill,",Margot Robbie",",Matthew McConaughey",Comedy,Crime
13,tt0468569,The Dark Knight,2008,PG-13,152 min,Action,9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,/name/nm0634240/,Christian Bale,"/name/nm0000288/,/name/nm0005132/,/name/nm0001...",2675531,534858444,Heath Ledger,",Aaron Eckhart",",Michael Caine",Crime,Drama
14,tt0144084,American Psycho,2000,R,102 min,Crime,7.6,A wealthy New York City investment banking exe...,Mary Harron,/name/nm0366004/,Christian Bale,"/name/nm0000288/,/name/nm0857620/,/name/nm0524...",636221,15070285,Justin Theroux,",Josh Lucas",",Bill Sage",Drama,Horror
21,tt2911666,John Wick,2014,R,101 min,Action,7.4,An ex-hit-man comes out of retirement to track...,"Chad Stahelski, \nDavid Leitch",/name/nm0821432/,Keanu Reeves,"/name/nm0500610/,/name/nm0000206/,/name/nm0638...",644257,43037835,Michael Nyqvist,",Alfie Allen",",Willem Dafoe",Crime,Thriller
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,tt0407551,Autumn,2004,Unrated,110 min,Crime,6.1,Jean-Pierre is a hit man in Paris. He wants to...,Ra'up McGee,/name/nm1098586/,Laurent Lucas,"/name/nm0524217/,/name/nm0001393/,/name/nm0738...",139,10529,Irène Jacob,",Benjamin Rolland",",Dinara Drukarova",Drama,None
18320,tt2633826,Mumbai Mirror,2013,Not Rated,136 min,Action,5.1,A cop tries to expose a drug racket run by a g...,Ankush Bhatt,/name/nm3932593/,Sachiin Joshi,"/name/nm4580164/,/name/nm0695177/,/name/nm2545...",126,4550,Prakash Raj,",Vimala Raman",",Mukesh Agrohari",Crime,Thriller
18402,tt0245904,Forgive Me Father,2001,R,138 min,Crime,5.2,"A ruthless mob hit-man, has been living a new ...",Ivan Rogers,/name/nm0004677/,Ivan Rogers,"/name/nm0004677/,/name/nm0464620/,/name/nm0621...",94,438906,Rich Komenich,",Charles Napier",",Chris Elbert",Drama,Mystery
23621,tt5114176,I Recorded a Murder!,2016,Not Rated,100 min,Crime,6.3,A filmmaker who is trying to finish his short ...,Manny Velazquez,/name/nm3220274/,Adriana Carradero,"/name/nm7204020/,/name/nm6885929/,/name/nm6646...",33,1500,Deliza Alcozer,",Mildred Amador",",Naudia Brown",Drama,Horror


In [79]:
# Drop 'actor_3', 'actor_4, Genre 2, Genre 3 columns
new_order_imdb_movies_df = ['Movie Title', 'Year', 'Domestic Revenue ($)', 'Genre', 'Main Actor', 'Supporting Actor','Director', 'Film Rating', 'Runtime', 'Votes']
new_order_imdb_movies_df = filtered_imdb_movies_by_genre_df[new_order_imdb_movies_df]
new_order_imdb_movies_df.reset_index(drop=True)
new_order_imdb_movies_df

,Movie Title,Year,Domestic Revenue ($),Genre,Main Actor,Supporting Actor,Director,Film Rating,Runtime,Votes
7,Knives Out,2019,165359751,Comedy,Daniel Craig,Chris Evans,Rian Johnson,PG-13,130 min,705622
9,The Wolf of Wall Street,2013,116900694,Biography,Leonardo DiCaprio,Jonah Hill,Martin Scorsese,R,180 min,1430476
13,The Dark Knight,2008,534858444,Action,Christian Bale,Heath Ledger,Christopher Nolan,PG-13,152 min,2675531
14,American Psycho,2000,15070285,Crime,Christian Bale,Justin Theroux,Mary Harron,R,102 min,636221
21,John Wick,2014,43037835,Action,Keanu Reeves,Michael Nyqvist,"Chad Stahelski, \nDavid Leitch",R,101 min,644257
...,...,...,...,...,...,...,...,...,...,...
18182,Autumn,2004,10529,Crime,Laurent Lucas,Irène Jacob,Ra'up McGee,Unrated,110 min,139
18320,Mumbai Mirror,2013,4550,Action,Sachiin Joshi,Prakash Raj,Ankush Bhatt,Not Rated,136 min,126
18402,Forgive Me Father,2001,438906,Crime,Ivan Rogers,Rich Komenich,Ivan Rogers,R,138 min,94
23621,I Recorded a Murder!,2016,1500,Crime,Adriana Carradero,Deliza Alcozer,Manny Velazquez,Not Rated,100 min,33


In [80]:
# Reset index
new_order_imdb_movies_df = new_order_imdb_movies_df.reset_index(drop=True)
new_order_imdb_movies_df.index = new_order_imdb_movies_df.index + 1
new_order_imdb_movies_df

,Movie Title,Year,Domestic Revenue ($),Genre,Main Actor,Supporting Actor,Director,Film Rating,Runtime,Votes
1,Knives Out,2019,165359751,Comedy,Daniel Craig,Chris Evans,Rian Johnson,PG-13,130 min,705622
2,The Wolf of Wall Street,2013,116900694,Biography,Leonardo DiCaprio,Jonah Hill,Martin Scorsese,R,180 min,1430476
3,The Dark Knight,2008,534858444,Action,Christian Bale,Heath Ledger,Christopher Nolan,PG-13,152 min,2675531
4,American Psycho,2000,15070285,Crime,Christian Bale,Justin Theroux,Mary Harron,R,102 min,636221
5,John Wick,2014,43037835,Action,Keanu Reeves,Michael Nyqvist,"Chad Stahelski, \nDavid Leitch",R,101 min,644257
...,...,...,...,...,...,...,...,...,...,...
1346,Autumn,2004,10529,Crime,Laurent Lucas,Irène Jacob,Ra'up McGee,Unrated,110 min,139
1347,Mumbai Mirror,2013,4550,Action,Sachiin Joshi,Prakash Raj,Ankush Bhatt,Not Rated,136 min,126
1348,Forgive Me Father,2001,438906,Crime,Ivan Rogers,Rich Komenich,Ivan Rogers,R,138 min,94
1349,I Recorded a Murder!,2016,1500,Crime,Adriana Carradero,Deliza Alcozer,Manny Velazquez,Not Rated,100 min,33


In [81]:
rename_imdb_movies_df = new_order_imdb_movies_df.rename(columns={'Year':'Release Year', 'Main Actor':'Leading Actor'})
rename_imdb_movies_df

,Movie Title,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Director,Film Rating,Runtime,Votes
1,Knives Out,2019,165359751,Comedy,Daniel Craig,Chris Evans,Rian Johnson,PG-13,130 min,705622
2,The Wolf of Wall Street,2013,116900694,Biography,Leonardo DiCaprio,Jonah Hill,Martin Scorsese,R,180 min,1430476
3,The Dark Knight,2008,534858444,Action,Christian Bale,Heath Ledger,Christopher Nolan,PG-13,152 min,2675531
4,American Psycho,2000,15070285,Crime,Christian Bale,Justin Theroux,Mary Harron,R,102 min,636221
5,John Wick,2014,43037835,Action,Keanu Reeves,Michael Nyqvist,"Chad Stahelski, \nDavid Leitch",R,101 min,644257
...,...,...,...,...,...,...,...,...,...,...
1346,Autumn,2004,10529,Crime,Laurent Lucas,Irène Jacob,Ra'up McGee,Unrated,110 min,139
1347,Mumbai Mirror,2013,4550,Action,Sachiin Joshi,Prakash Raj,Ankush Bhatt,Not Rated,136 min,126
1348,Forgive Me Father,2001,438906,Crime,Ivan Rogers,Rich Komenich,Ivan Rogers,R,138 min,94
1349,I Recorded a Murder!,2016,1500,Crime,Adriana Carradero,Deliza Alcozer,Manny Velazquez,Not Rated,100 min,33


In [82]:
merge_mojo_imbd_movies_2000_2020 = pd.merge(merge_mojo_movies_data_clean_df, rename_imdb_movies_df[['Movie Title', 'Release Year', 'Domestic Revenue ($)', 'Genre',	'Leading Actor', 'Supporting Actor', 'Film Rating']], on=['Movie Title','Release Year', 'Domestic Revenue ($)', 'Genre', 'Leading Actor', 'Supporting Actor', 'Film Rating'], how='left')
merge_mojo_imbd_movies_2000_2020

,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13
1,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13
2,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R
3,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R
4,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13
...,...,...,...,...,...,...,...,...
2390,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R
2391,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13
2392,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R
2393,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG


In [83]:
merge_mojo_imbd_movies_2000_2020.isnull().sum()

Movie Title             0
Release Date            0
Release Year            0
Domestic Revenue ($)    0
Genre                   0
Leading Actor           0
Supporting Actor        0
Film Rating             0
dtype: int64

In [84]:
merge_mojo_imbd_movies_2000_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395 entries, 0 to 2394
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Movie Title           2395 non-null   object
 1   Release Date          2395 non-null   object
 2   Release Year          2395 non-null   int64 
 3   Domestic Revenue ($)  2395 non-null   int64 
 4   Genre                 2395 non-null   object
 5   Leading Actor         2395 non-null   object
 6   Supporting Actor      2395 non-null   object
 7   Film Rating           2395 non-null   object
dtypes: int64(2), object(6)
memory usage: 149.8+ KB


In [85]:
merge_mojo_imbd_movies_2000_2020['Combined'] = merge_mojo_imbd_movies_2000_2020['Release Date'].astype(str) + ' ' + merge_mojo_imbd_movies_2000_2020['Release Year'].astype(str)

merge_mojo_imbd_movies_2000_2020

,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Combined
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13,September 21 2001
1,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,July 6 2005
2,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R,October 25 2002
3,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R,October 19 2001
4,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13,December 19 2001
...,...,...,...,...,...,...,...,...,...
2390,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R,November 21 2019
2391,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,November 21 2019
2392,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,March 29 2019
2393,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG,November 7 2019


In [86]:
merge_mojo_imbd_movies_2000_2020['Combined'] = pd.to_datetime(merge_mojo_imbd_movies_2000_2020['Combined'], errors='coerce')
merge_mojo_imbd_movies_2000_2020

,Movie Title,Release Date,Release Year,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Combined
0,Glitter,September 21,2001,4274407,Drama,Mariah Carey,Eric Benét,PG-13,2001-09-21
1,Fantastic Four,July 6,2005,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,2005-07-06
2,Frida,October 25,2002,25885000,Biography,Salma Hayek,Alfred Molina,R,2002-10-25
3,From Hell,October 19,2001,31602566,Horror,Johnny Depp,Heather Graham,R,2001-10-19
4,The Lord of the Rings: The Fellowship of the Ring,December 19,2001,315544750,Action,Elijah Wood,Ian McKellen,PG-13,2001-12-19
...,...,...,...,...,...,...,...,...,...
2390,21 Bridges,November 21,2019,28539757,Action,Chadwick Boseman,Sienna Miller,R,2019-11-21
2391,Knives Out,November 21,2019,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,2019-11-21
2392,Unplanned,March 29,2019,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,2019-03-29
2393,Playing with Fire,November 7,2019,44451847,Comedy,John Cena,Keegan-Michael Key,PG,2019-11-07


In [87]:
new_merged_df = merge_mojo_imbd_movies_2000_2020.drop(columns=['Release Date', 'Release Year'])
new_merged_df

,Movie Title,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Combined
0,Glitter,4274407,Drama,Mariah Carey,Eric Benét,PG-13,2001-09-21
1,Fantastic Four,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,2005-07-06
2,Frida,25885000,Biography,Salma Hayek,Alfred Molina,R,2002-10-25
3,From Hell,31602566,Horror,Johnny Depp,Heather Graham,R,2001-10-19
4,The Lord of the Rings: The Fellowship of the Ring,315544750,Action,Elijah Wood,Ian McKellen,PG-13,2001-12-19
...,...,...,...,...,...,...,...
2390,21 Bridges,28539757,Action,Chadwick Boseman,Sienna Miller,R,2019-11-21
2391,Knives Out,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,2019-11-21
2392,Unplanned,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,2019-03-29
2393,Playing with Fire,44451847,Comedy,John Cena,Keegan-Michael Key,PG,2019-11-07


In [88]:
#rename Combined to Release Date
new_merged_df = new_merged_df.rename(columns={'Combined':'Release Date'})
new_merged_df

,Movie Title,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Release Date
0,Glitter,4274407,Drama,Mariah Carey,Eric Benét,PG-13,2001-09-21
1,Fantastic Four,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,2005-07-06
2,Frida,25885000,Biography,Salma Hayek,Alfred Molina,R,2002-10-25
3,From Hell,31602566,Horror,Johnny Depp,Heather Graham,R,2001-10-19
4,The Lord of the Rings: The Fellowship of the Ring,315544750,Action,Elijah Wood,Ian McKellen,PG-13,2001-12-19
...,...,...,...,...,...,...,...
2390,21 Bridges,28539757,Action,Chadwick Boseman,Sienna Miller,R,2019-11-21
2391,Knives Out,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,2019-11-21
2392,Unplanned,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,2019-03-29
2393,Playing with Fire,44451847,Comedy,John Cena,Keegan-Michael Key,PG,2019-11-07


In [89]:
# approximate_box_office_df = approximate_city_box_office_with_theaters(new_order_imdb_movies_df, "Miami")
# display(approximate_box_office_df)

In [90]:
# approximate_box_office_df.to_csv("approximate_box_office_miami.csv", index=False)

In [91]:
seattle_gross_data_df = pd.read_csv('Resources/approximate_box_office_seattle.csv')
new_york_gross_data_df = pd.read_csv('Resources/approximate_box_office_new_york.csv')
los_angeles_gross_data_df = pd.read_csv('Resources/approximate_box_office_los_angeles.csv')
phoenix_gross_data_df = pd.read_csv('Resources/approximate_box_office_phoenix.csv')
houston_gross_data_df = pd.read_csv('Resources/approximate_box_office_houston.csv')
chicago_gross_data_df = pd.read_csv('Resources/approximate_box_office_chicago.csv')
miami_gross_data_df = pd.read_csv('Resources/approximate_box_office_miami.csv')


In [92]:
rename_seattle_gross_data_df = seattle_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Seattle Revenue ($)'})
new_seattle_df = rename_seattle_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_seattle_df

,Movie Title,Seattle Revenue ($)
0,Knives Out,"$105,842.99"
1,The Wolf of Wall Street,"$74,825.46"
2,The Dark Knight,"$342,350.63"
3,American Psycho,"$9,646.14"
4,John Wick,"$27,547.53"
...,...,...
1345,Autumn,$6.74
1346,Mumbai Mirror,$2.91
1347,Forgive Me Father,$280.93
1348,I Recorded a Murder!,$0.96


In [93]:
rename_new_york_gross_data_df = new_york_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'New York Revenue ($)'})
new_new_york_df = rename_new_york_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_new_york_df

,Movie Title,New York Revenue ($)
0,Knives Out,"$1,397,345.41"
1,The Wolf of Wall Street,"$987,850.11"
2,The Dark Knight,"$4,519,733.43"
3,American Psycho,"$127,348.97"
4,John Wick,"$363,684.16"
...,...,...
1345,Autumn,$88.97
1346,Mumbai Mirror,$38.45
1347,Forgive Me Father,"$3,708.90"
1348,I Recorded a Murder!,$12.68


In [94]:
rename_los_angeles_gross_data_df = los_angeles_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Los Angeles Revenue ($)'})
new_los_angeles_df = rename_los_angeles_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_los_angeles_df

,Movie Title,Los Angeles Revenue ($)
0,Knives Out,"$648,346.54"
1,The Wolf of Wall Street,"$458,347.09"
2,The Dark Knight,"$2,097,086.00"
3,American Psycho,"$59,087.94"
4,John Wick,"$168,743.79"
...,...,...
1345,Autumn,$41.28
1346,Mumbai Mirror,$17.84
1347,Forgive Me Father,"$1,720.87"
1348,I Recorded a Murder!,$5.88


In [95]:
rename_phoenix_gross_data_df = phoenix_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Phoenix Revenue ($)'})
new_phoenix_df = rename_phoenix_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_phoenix_df

,Movie Title,Phoenix Revenue ($)
0,Knives Out,"$256,040.07"
1,The Wolf of Wall Street,"$181,006.94"
2,The Dark Knight,"$828,165.23"
3,American Psycho,"$23,334.56"
4,John Wick,"$66,639.01"
...,...,...
1345,Autumn,$16.30
1346,Mumbai Mirror,$7.05
1347,Forgive Me Father,$679.59
1348,I Recorded a Murder!,$2.32


In [96]:
rename_houston_gross_data_df = houston_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Houston Revenue ($)'})
new_houston_df = rename_houston_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_houston_df

,Movie Title,Houston Revenue ($)
0,Knives Out,"$361,130.61"
1,The Wolf of Wall Street,"$255,300.45"
2,The Dark Knight,"$1,168,082.04"
3,American Psycho,"$32,912.13"
4,John Wick,"$93,990.70"
...,...,...
1345,Autumn,$22.99
1346,Mumbai Mirror,$9.94
1347,Forgive Me Father,$958.53
1348,I Recorded a Murder!,$3.28


In [97]:
rename_chicago_gross_data_df = chicago_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Chicago Revenue ($)'})
new_chicago_df = rename_chicago_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_chicago_df

,Movie Title,Chicago Revenue ($)
0,Knives Out,"$471,778.22"
1,The Wolf of Wall Street,"$333,522.52"
2,The Dark Knight,"$1,525,973.28"
3,American Psycho,"$42,996.15"
4,John Wick,"$122,788.73"
...,...,...
1345,Autumn,$30.04
1346,Mumbai Mirror,$12.98
1347,Forgive Me Father,"$1,252.22"
1348,I Recorded a Murder!,$4.28


In [98]:
rename_miami_gross_data_df = miami_gross_data_df.rename(columns={'movie':'Movie Title', 'estimated_city_gross':'Miami Revenue ($)'})
new_miami_df = rename_miami_gross_data_df.drop(['city', 'theater_count'], axis=1)
new_miami_df

,Movie Title,Miami Revenue ($)
0,Knives Out,"$66,711.18"
1,The Wolf of Wall Street,"$47,161.32"
2,The Dark Knight,"$215,778.26"
3,American Psycho,"$6,079.81"
4,John Wick,"$17,362.78"
...,...,...
1345,Autumn,$4.25
1346,Mumbai Mirror,$1.84
1347,Forgive Me Father,$177.07
1348,I Recorded a Murder!,$0.61


In [118]:
miami_duplicates = new_miami_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

new_york_duplicates = new_new_york_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

chicago_duplicates = new_chicago_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

phoenix_duplicates = new_phoenix_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

houston_duplicates = new_houston_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

los_angeles_duplicates = new_los_angeles_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

seattle_duplicates = new_seattle_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')


Total duplicate rows: 0
Total duplicate rows: 0
Total duplicate rows: 0
Total duplicate rows: 0
Total duplicate rows: 0
Total duplicate rows: 0
Total duplicate rows: 0


new_merged_df, new_miami_df, new_chicago_df, new_houston_df, new_phoenix_df, new_los_angeles_df, new_new_york_df, new_seattle_df

[['Movie Title', 'Release Year', 'Domestic Revenue ($)', 'Genre',	'Leading Actor', 'Supporting Actor', 'Film Rating']], on=['Movie Title','Release Year', 'Domestic Revenue ($)', 'Genre', 'Leading Actor', 'Supporting Actor', 'Film Rating'], how='left')

In [134]:
merge_movies_cities_df = pd.merge(new_merged_df, new_miami_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_chicago_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_houston_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_phoenix_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_los_angeles_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_new_york_df, on='Movie Title', how='left')
merge_movies_cities_df = pd.merge(merge_movies_cities_df, new_seattle_df, on='Movie Title', how='left')


merge_movies_cities_df.drop_duplicates(subset=['Movie Title'], inplace=True)


In [135]:
merge_movies_cities_df

,Movie Title,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Release Date,Miami Revenue ($),Chicago Revenue ($),Houston Revenue ($),Phoenix Revenue ($),Los Angeles Revenue ($),New York Revenue ($),Seattle Revenue ($)
0,Glitter,4274407,Drama,Mariah Carey,Eric Benét,PG-13,2001-09-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fantastic Four,154696080,Action,Ioan Gruffudd,Michael Chiklis,PG-13,2005-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Frida,25885000,Biography,Salma Hayek,Alfred Molina,R,2002-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,From Hell,31602566,Horror,Johnny Depp,Heather Graham,R,2001-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Lord of the Rings: The Fellowship of the Ring,315544750,Action,Elijah Wood,Ian McKellen,PG-13,2001-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3152,21 Bridges,28539757,Action,Chadwick Boseman,Sienna Miller,R,2019-11-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3153,Knives Out,165363234,Comedy,Daniel Craig,Chris Evans,PG-13,2019-11-21,"$66,711.18","$471,778.22","$361,130.61","$256,040.07","$648,346.54","$1,397,345.41","$105,842.99"
3154,Unplanned,19005109,Biography,Ashley Bratcher,Brooks Ryan,R,2019-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3155,Playing with Fire,44451847,Comedy,John Cena,Keegan-Michael Key,PG,2019-11-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
merge_movies_cities_df.isnull().sum()

Movie Title                   0
Domestic Revenue ($)          0
Genre                         0
Leading Actor                 0
Supporting Actor              0
Film Rating                   0
Release Date                  0
Miami Revenue ($)          1939
Chicago Revenue ($)        1939
Houston Revenue ($)        1939
Phoenix Revenue ($)        1939
Los Angeles Revenue ($)    1939
New York Revenue ($)       1939
Seattle Revenue ($)        1939
dtype: int64

In [138]:
clean_movies_cities_df = merge_movies_cities_df.dropna()
clean_movies_cities_df.sort_values(by=['Movie Title','Release Date'], ascending=True, inplace=True)
clean_movies_cities_df.reset_index(drop=True, inplace=True)
clean_movies_cities_df

/var/folders/ts/dsv00vbn2ygg_1mgkl5xxg_c0000gn/T/ipykernel_5245/3757080463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movies_cities_df.sort_values(by=['Movie Title','Release Date'], ascending=True, inplace=True)


,Movie Title,Domestic Revenue ($),Genre,Leading Actor,Supporting Actor,Film Rating,Release Date,Miami Revenue ($),Chicago Revenue ($),Houston Revenue ($),Phoenix Revenue ($),Los Angeles Revenue ($),New York Revenue ($),Seattle Revenue ($)
0,10th & Wolf,54702,Crime,James Marsden,Giovanni Ribisi,R,2006-08-18,$21.58,$152.58,$116.80,$82.81,$209.69,$451.93,$34.23
1,15 Minutes,24403552,Action,Robert De Niro,Edward Burns,R,2001-03-09,"$9,845.14","$69,624.34","$53,295.13","$37,786.02","$95,682.04","$206,218.21","$15,620.15"
2,2 Fast 2 Furious,127154901,Action,Paul Walker,Tyrese Gibson,PG-13,2003-06-05,"$51,298.18","$362,778.20","$277,694.70","$196,884.37","$498,552.03","$1,074,501.61","$81,388.94"
3,21,81159365,Crime,Jim Sturgess,Kate Bosworth,PG-13,2008-03-28,"$32,742.17","$231,551.03","$177,244.65","$125,665.71","$318,211.61","$685,823.88","$51,948.25"
4,21 Grams,16290476,Crime,Sean Penn,Benicio Del Toro,R,2003-11-21,"$6,572.08","$46,477.40","$35,576.91","$25,223.88","$63,872.09","$137,659.99","$10,427.16"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,Whiteout,10275638,Action,Kate Beckinsale,Gabriel Macht,R,2009-09-11,"$4,145.51","$29,316.82","$22,441.06","$15,910.61","$40,288.97","$86,832.59","$6,577.20"
440,Wild Target,108589,Action,Bill Nighy,Emily Blunt,PG-13,2010-06-18,$47.28,$334.35,$255.93,$181.45,$459.48,$990.29,$75.01
441,Wind River,33800859,Crime,Kelsey Asbille,Jeremy Renner,R,2017-08-03,"$13,636.30","$96,435.25","$73,817.99","$52,336.64","$132,527.23","$285,628.61","$21,635.16"
442,Young Adam,767373,Crime,Ewan McGregor,Tilda Swinton,NC-17,2003-03-19,$309.53,"$2,188.97","$1,675.58","$1,187.98","$3,008.21","$6,483.44",$491.09


In [139]:
movies_cities_duplicates = clean_movies_cities_df.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

Total duplicate rows: 0
